In [1]:
import time
import json
import logging
import requests
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup

In [2]:
LOG_FILENAME = "lastlog.log"
JSON_FILENAME = "ncov_{ctime}.json".format(ctime=time.strftime('%Y%m%d-%H%M%S'))
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
with open(LOG_FILENAME, 'w') as f:
    f.writelines([LOG_FORMAT, '\n'])
logging.basicConfig(filename=LOG_FILENAME, level=logging.DEBUG, format=LOG_FORMAT)

logging.debug("This is a debug log.")
logging.info("This is a info log.")
logging.warning("This is a warning log.")
logging.error("This is a error log.")
logging.critical("This is a critical log.")

In [3]:
response = requests.get("https://ncov.dxy.cn/ncovh5/view/pneumonia")
HTML = response.text.encode(response.encoding).decode()
soup = BeautifulSoup(HTML)
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html lang="zh-cn" xmlns:layout="http://www.ultraq.net.nz/web/thymeleaf/layout"><head>
<link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/umi.bundle.css?t=1581162032931" rel="stylesheet"/>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
<meta content="#000000" name="theme-color"/>
<title>全国新冠肺炎疫情实时动态 - 丁香园·丁香医生</title>
<script>
        window.routerBase = "/ncovh5/view";
    </script>
<script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia~p__Pneumonia__recommend-list~p__Pneumonia__rumor-list~p__Pneumonia__timeline.async.5ccb549d.js"></script><script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia~p__Pneumonia__rumor-list.async.9184546f.js"></script><link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia.async.9f1a94c3.css" rel="stylesheet" type="text/css"/>

In [4]:
text = soup.contents[1].text
def parse_text(text):
    subtext = text[text.find("window.getAreaStat")::]
    subtext = subtext[subtext.find("[{")::]
    # print(subtext)
    num = 0
    chars = []
    for c in subtext:
        chars.append(c)
        if c == '[':
            num += 1
        if c == ']':
            num -= 1
        if num == 0:
            break
    return ''.join(chars)
counting_json = json.loads(parse_text(text))
print(counting_json)

[{'provinceName': '湖北省', 'provinceShortName': '湖北', 'confirmedCount': 27100, 'suspectedCount': 23638, 'curedCount': 1439, 'deadCount': 780, 'comment': '', 'locationId': 420000, 'cities': [{'cityName': '武汉', 'confirmedCount': 14982, 'suspectedCount': 0, 'curedCount': 877, 'deadCount': 608, 'locationId': 420100}, {'cityName': '孝感', 'confirmedCount': 2436, 'suspectedCount': 0, 'curedCount': 45, 'deadCount': 29, 'locationId': 420900}, {'cityName': '黄冈', 'confirmedCount': 2141, 'suspectedCount': 0, 'curedCount': 137, 'deadCount': 43, 'locationId': 421100}, {'cityName': '荆州', 'confirmedCount': 997, 'suspectedCount': 0, 'curedCount': 34, 'deadCount': 13, 'locationId': 421000}, {'cityName': '襄阳', 'confirmedCount': 988, 'suspectedCount': 0, 'curedCount': 40, 'deadCount': 7, 'locationId': 420600}, {'cityName': '随州', 'confirmedCount': 984, 'suspectedCount': 0, 'curedCount': 23, 'deadCount': 9, 'locationId': 421300}, {'cityName': '黄石', 'confirmedCount': 760, 'suspectedCount': 0, 'curedCount': 53, 

In [5]:
country_df = pd.read_json(json.dumps(counting_json))
country_df.to_json(open(JSON_FILENAME, 'w'))
country_df

,provinceName,provinceShortName,confirmedCount,suspectedCount,curedCount,deadCount,comment,locationId,cities
0,湖北省,湖北,27100,23638,1439,780,,420000,"[{'cityName': '武汉', 'confirmedCount': 14982, '..."
1,广东省,广东,1095,0,112,1,,440000,"[{'cityName': '深圳', 'confirmedCount': 354, 'su..."
2,浙江省,浙江,1075,0,173,0,,330000,"[{'cityName': '温州', 'confirmedCount': 448, 'su..."
3,河南省,河南,1033,0,131,6,参考卫健委统计口径，部分县区与地级市合并,410000,"[{'cityName': '信阳', 'confirmedCount': 205, 'su..."
4,湖南省,湖南,838,0,159,1,,430000,"[{'cityName': '长沙', 'confirmedCount': 205, 'su..."
5,安徽省,安徽,779,0,59,1,,340000,"[{'cityName': '合肥', 'confirmedCount': 136, 'su..."
6,江西省,江西,740,0,72,0,,360000,"[{'cityName': '南昌', 'confirmedCount': 179, 'su..."
7,江苏省,江苏,468,0,51,0,,320000,"[{'cityName': '苏州', 'confirmedCount': 75, 'sus..."
8,重庆市,重庆,446,0,39,2,,500000,"[{'cityName': '万州区', 'confirmedCount': 84, 'su..."
9,山东省,山东,416,0,44,0,,370000,"[{'cityName': '青岛', 'confirmedCount': 46, 'sus..."


In [6]:
provinces_df = dict()
for idx in country_df.index:
    se = country_df.iloc[idx]
    if se['cities']:
        provinces_df[se['provinceName']] = pd.read_json(json.dumps(se['cities']))
pprint(provinces_df)

{'上海市':    cityName  confirmedCount  suspectedCount  curedCount  deadCount  locationId
0    外地来沪人员              97               0          20          1          -1
1      浦东新区              56               0           8          0      310115
2       宝山区              17               0           0          0      310113
3       静安区              16               0           3          0      310106
4       徐汇区              16               0           1          0      310104
5       闵行区              14               0           2          0      310112
6       松江区              14               0           0          0      310117
7       长宁区              11               0           1          0      310105
8       奉贤区               9               0           1          0      310120
9       普陀区               8               0           0          0      310107
10      虹口区               7               0           1          0      310109
11      杨浦区               7               0 

 '湖北省':    cityName  confirmedCount  suspectedCount  curedCount  deadCount  locationId
0        武汉           14982               0         877        608      420100
1        孝感            2436               0          45         29      420900
2        黄冈            2141               0         137         43      421100
3        荆州             997               0          34         13      421000
4        襄阳             988               0          40          7      420600
5        随州             984               0          23          9      421300
6        黄石             760               0          53          2      420200
7        宜昌             711               0          36          8      420500
8        荆门             663               0          48         19      420800
9        鄂州             639               0          42         21      420700
10       咸宁             493               0          23          4      421200
11       十堰             467               0 